In [1]:
import os
import sys
import collections

In [2]:
sys.path.extend(['..'])

In [3]:
sys.path

['/home/b/jchun/Documents/Envs/Lycastus-Py3/lib/python35.zip',
 '/home/b/jchun/Documents/Envs/Lycastus-Py3/lib/python3.5',
 '/home/b/jchun/Documents/Envs/Lycastus-Py3/lib/python3.5/plat-linux',
 '/home/b/jchun/Documents/Envs/Lycastus-Py3/lib/python3.5/lib-dynload',
 '/usr/local/lib/python3.5',
 '/usr/local/lib/python3.5/plat-linux',
 '',
 '/home/b/jchun/Documents/Envs/Lycastus-Py3/lib/python3.5/site-packages',
 '/home/b/jchun/Documents/Envs/Lycastus-Py3/lib/python3.5/site-packages/IPython/extensions',
 '/home/b/jchun/.ipython',
 '..']

In [4]:
import numpy as np
import tensorflow as tf

In [5]:
import drs
import modeling
from run_classifier import convert_single_example, model_fn_builder, InputExample
import tokenization
import attention

In [6]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [7]:
# flags
model_dir  = "../drs/output_implicit_2"
label_list = drs.get_senses_with_degree(2)

In [8]:
# bert paths
bert_config_file = "../uncased_L-12_H-768_A-12/bert_config.json"
bert_init_ckpt   = "../uncased_L-12_H-768_A-12/bert_model.ckpt"
bert_vocab_file  = "../uncased_L-12_H-768_A-12/vocab.txt"

In [9]:
# experiment settings
seq_length = 128
num_hidden_layers=12
num_attention_heads=12

In [10]:
# tokenizer
tokenizer  = tokenization.FullTokenizer(
      vocab_file=bert_vocab_file,
      do_lower_case=True)

In [11]:
bert_config = modeling.BertConfig.from_json_file(bert_config_file)

In [12]:
os.path.exists(model_dir)

True

In [13]:
label_list

['Contingency.Condition',
 'Expansion.Alternative',
 'Temporal.Asynchronous',
 'Comparison.Contrast',
 'Expansion.Instantiation',
 'Expansion.Exception',
 'Comparison.Concession',
 'Expansion.Conjunction',
 'Comparison',
 'Temporal',
 'Expansion.Restatement',
 'EntRel',
 'Expansion',
 'Contingency',
 'Temporal.Synchrony',
 'Contingency.Cause']

In [14]:
model_fn = model_fn_builder(
      bert_config=bert_config,
      num_labels=len(label_list),
      init_checkpoint=bert_init_ckpt,
      use_one_hot_embeddings=False,
      learning_rate=None,
      num_train_steps=None,
      num_warmup_steps=None)

In [15]:
run_config = tf.contrib.tpu.RunConfig(
      cluster=None,
      model_dir=model_dir)

In [16]:
model = tf.contrib.tpu.TPUEstimator(
    use_tpu=False,
    model_fn=model_fn,
    config=run_config)

INFO:tensorflow:Using config: {'_device_fn': None, '_task_type': 'worker', '_task_id': 0, '_experimental_distribute': None, '_protocol': None, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_tf_random_seed': None, '_evaluation_master': '', '_num_worker_replicas': 1, '_cluster': None, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=None, num_cores_per_replica=None, per_host_input_for_training=2, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None), '_log_step_count_steps': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc66b0c12e8>, '_eval_distribute': None, '_model_dir': '../drs/output_implicit_2', '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_train_distribute': None, '_is_chief': True, '_master':

In [17]:
model.get_variable_names()

['bert/embeddings/LayerNorm/beta',
 'bert/embeddings/LayerNorm/beta/adam_m',
 'bert/embeddings/LayerNorm/beta/adam_v',
 'bert/embeddings/LayerNorm/gamma',
 'bert/embeddings/LayerNorm/gamma/adam_m',
 'bert/embeddings/LayerNorm/gamma/adam_v',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/position_embeddings/adam_m',
 'bert/embeddings/position_embeddings/adam_v',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/token_type_embeddings/adam_m',
 'bert/embeddings/token_type_embeddings/adam_v',
 'bert/embeddings/word_embeddings',
 'bert/embeddings/word_embeddings/adam_m',
 'bert/embeddings/word_embeddings/adam_v',
 'bert/encoder/layer_0/attention/output/LayerNorm/beta',
 'bert/encoder/layer_0/attention/output/LayerNorm/beta/adam_m',
 'bert/encoder/layer_0/attention/output/LayerNorm/beta/adam_v',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma/adam_m',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma

In [18]:
def input_fn_builder(input_):
    name_to_features = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "label_ids": tf.FixedLenFeature([], tf.int64),
      "is_real_example": tf.FixedLenFeature([], tf.int64),
      }
    
    def input_fn(params):
        return tf.parse_single_example(input_, name_to_features)
    return input_fn

In [19]:
input_ = "I went too I knew little of where, however."

In [20]:
input_example = InputExample(guid=0, text_a="I went too", text_b="I knew little of where, however.", label='EntRel')

In [21]:
feature = convert_single_example(0, input_example, label_list, seq_length, tokenizer)

INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: 0
INFO:tensorflow:tokens: [CLS] i went too [SEP] i knew little of where , however . [SEP]
INFO:tensorflow:input_ids: 101 1045 2253 2205 102 1045 2354 2210 1997 2073 1010 2174 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [22]:
def create_int_feature(values):
    f = tf.train.Feature(int64_list=tf.train.Int64List(value=list(values)))
    return f

In [23]:
features = collections.OrderedDict()
features["input_ids"] = create_int_feature(feature.input_ids)
features["input_mask"] = create_int_feature(feature.input_mask)
features["segment_ids"] = create_int_feature(feature.segment_ids)
features["label_ids"] = create_int_feature([feature.label_id])
features["is_real_example"] = create_int_feature(
    [int(feature.is_real_example)])

In [24]:
tf_example = tf.train.Example(features=tf.train.Features(feature=features))

In [25]:
viz_pred_file = os.path.join(model_dir, "viz_pred.tf_record")

In [26]:
writer = tf.python_io.TFRecordWriter(viz_pred_file)

In [27]:
writer.write(tf_example.SerializeToString())

In [28]:
writer.close()

In [29]:
def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""
    
    name_to_features = {
          "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
          "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
          "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
          "label_ids": tf.FixedLenFeature([], tf.int64),
          "is_real_example": tf.FixedLenFeature([], tf.int64),
        }

    def input_fn(params):
        """The actual input function."""
        d = tf.data.TFRecordDataset(input_file)
        d = d.apply(
            tf.contrib.data.map_and_batch(
                lambda record: tf.parse_single_example(record, name_to_features),
                batch_size=1,
                drop_remainder=drop_remainder))
        return d

    return input_fn

In [30]:
predict_input_fn = file_based_input_fn_builder(
        input_file=viz_pred_file,
        seq_length=seq_length,
        is_training=False,
        drop_remainder=False)

In [31]:
result = model.predict(input_fn=predict_input_fn)

In [32]:
for (i, prediction) in enumerate(result):
    probabilities = prediction["probabilities"]
    label = label_list[np.argmax(probabilities)]

Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name =

INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKP

INFO:tensorflow:  name = bert/encoder/layer_8/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_8/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/

In [33]:
label

'Comparison.Contrast'

In [34]:
model.get_variable_names()

['bert/embeddings/LayerNorm/beta',
 'bert/embeddings/LayerNorm/beta/adam_m',
 'bert/embeddings/LayerNorm/beta/adam_v',
 'bert/embeddings/LayerNorm/gamma',
 'bert/embeddings/LayerNorm/gamma/adam_m',
 'bert/embeddings/LayerNorm/gamma/adam_v',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/position_embeddings/adam_m',
 'bert/embeddings/position_embeddings/adam_v',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/token_type_embeddings/adam_m',
 'bert/embeddings/token_type_embeddings/adam_v',
 'bert/embeddings/word_embeddings',
 'bert/embeddings/word_embeddings/adam_m',
 'bert/embeddings/word_embeddings/adam_v',
 'bert/encoder/layer_0/attention/output/LayerNorm/beta',
 'bert/encoder/layer_0/attention/output/LayerNorm/beta/adam_m',
 'bert/encoder/layer_0/attention/output/LayerNorm/beta/adam_v',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma/adam_m',
 'bert/encoder/layer_0/attention/output/LayerNorm/gamma

In [35]:
model.get_variable_value('bert/encoder/layer_9/attention/self/key/kernel')

array([[-0.01383783,  0.00570357, -0.04701683, ..., -0.04125093,
        -0.04221822,  0.03177653],
       [ 0.00646215, -0.04762628, -0.02844496, ...,  0.01216382,
         0.00347079,  0.02835898],
       [-0.05348697, -0.05752722,  0.01192973, ...,  0.0012023 ,
        -0.03955814, -0.04730618],
       ...,
       [ 0.04078497,  0.03955465, -0.05358118, ...,  0.03683624,
         0.02867581,  0.00650171],
       [ 0.04629496, -0.06310652, -0.04017214, ...,  0.11711372,
        -0.05839688,  0.01878156],
       [ 0.00461925,  0.01636456, -0.04651957, ..., -0.0224819 ,
        -0.04020511, -0.00791032]], dtype=float32)

# Now for Viz

In [36]:
import json
import os

In [37]:
import IPython.display as display

In [38]:
vis_html = """
  <span style="user-select:none">
    Layer: <select id="layer"></select>
    Attention: <select id="att_type">
      <option value="all">All</option>
      <option value="a">Sentence A self-attention</option>
      <option value="b">Sentence B self-attention</option>
      <option value="ab">Sentence A -> Sentence B</option>
      <option value="ba">Sentence B -> Sentence A</option>
    </select>
  </span>
  <div id='vis'></div>
"""

In [39]:
__location__ = os.path.realpath(
    os.path.join(os.getcwd(), os.path.dirname("__file__")))
vis_js = open(os.path.join(__location__, 'attention.js')).read()

In [40]:
def get_att_mats(bert_model):
    self_attentions = []
    
    layer_format = 'bert/encoder/layer_{}/attention/self/key/kernel'
    for i in range(num_hidden_layers):
        i_layer = layer_format.format(i)
        self_attention = bert_model.get_variable_value(i_layer)
        self_attentions.append(self_attention)
    return self_attentions

In [41]:
self_attentions = get_att_mats(model)

In [42]:
len(self_attentions), len(self_attentions[0])

(12, 768)

In [43]:
self_attentions[0].shape

(768, 768)

In [46]:
_attn = self_attentions[0]

In [52]:
_attn

array([[ 0.0081959 ,  0.02625902, -0.01941409, ...,  0.02095003,
        -0.053547  ,  0.00595329],
       [-0.03752267, -0.06333707, -0.01727781, ..., -0.01776117,
         0.02126005, -0.04799785],
       [ 0.01843449,  0.01703105, -0.03340731, ..., -0.01087551,
        -0.01802054,  0.02423437],
       ...,
       [ 0.0151049 , -0.01798486, -0.02701281, ...,  0.01855043,
        -0.01006883, -0.00478966],
       [-0.01491464, -0.04380539, -0.04236244, ..., -0.05464846,
        -0.01638294, -0.00108903],
       [-0.00956593,  0.01627923, -0.01053867, ...,  0.08322829,
        -0.06601245, -0.00152366]], dtype=float32)

In [44]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min'
  }
});

<IPython.core.display.Javascript object>

In [45]:
attention.show(input_, input_, self_attentions)

A, B
slice(0, 43, None) slice(43, 86, None)
Layer ATTN
(768, 768)
[[ 0.0081959   0.02625902 -0.01941409 ...  0.02095003 -0.053547
   0.00595329]
 [-0.03752267 -0.06333707 -0.01727781 ... -0.01776117  0.02126005
  -0.04799785]
 [ 0.01843449  0.01703105 -0.03340731 ... -0.01087551 -0.01802054
   0.02423437]
 ...
 [ 0.0151049  -0.01798486 -0.02701281 ...  0.01855043 -0.01006883
  -0.00478966]
 [-0.01491464 -0.04380539 -0.04236244 ... -0.05464846 -0.01638294
  -0.00108903]
 [-0.00956593  0.01627923 -0.01053867 ...  0.08322829 -0.06601245
  -0.00152366]]


IndexError: too many indices for array